In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 50)

In [ ]:
os.getcwd()

In [ ]:
df_generic1 = pd.read_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/CommunityProcess/Generic Tweet - Sheet1.csv')
df_generic2 = pd.read_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/CommunityProcess/Generic Tweet - Sheet2.csv')

In [ ]:
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_cluster_small =pd.read_csv('All_Cluster_Tweet_Topic.csv')

In [ ]:
df_gen = list(set(df_cluster_small[df_cluster_small.Topic_ID3.isin(df_generic1['Generic Message'].tolist())].Cluster_ID.tolist()))

In [ ]:
df_with_nan_only = df_generic2.dropna(axis=1, how='all')

In [ ]:
rows_with_nan =  df_generic2.dropna(subset= df_generic2.columns[ df_generic2.isna().any()], how='any')

In [ ]:
rows_with_nan.Campaign

In [ ]:
df_generic2_updated = df_generic2[~df_generic2.Campaign.isin(rows_with_nan.Campaign)]

In [ ]:
df_gen_up = df_gen + df_generic2_updated.Campaign.tolist()


In [ ]:
len(list(set(df_gen_up)))

In [ ]:
df_all_cluster = pd.read_csv('Cluster_Tweet_Final.csv')

In [ ]:
df_all_cluster[df_all_cluster.Cluster_ID.isin(df_gen_up)]

In [ ]:
df_gen_up.append(8694)

In [ ]:
df_all_cluster_refined = df_all_cluster[~df_all_cluster.Cluster_ID.isin(df_gen_up)]

In [ ]:
df_all_cluster_refined.to_csv('Cluster_Tweet_Without_Generic.csv')

In [ ]:
import networkx as nx
from itertools import combinations
import pandas as pd


G = nx.Graph()

user_clusters = df_all_cluster_refined.groupby('User_ID')['Cluster_ID'].unique().to_dict()

for pair in combinations(user_clusters.keys(), 2):
    user1, user2 = pair
    common_clusters = set(user_clusters[user1]).intersection(user_clusters[user2])
    if len(common_clusters) >= 5:
        weight = len(common_clusters)
        G.add_edge(user1, user2, weight=weight)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

for u, v, data in G.edges(data=True):
    print(f"Edge ({u}, {v}): Weight = {data['weight']}")

In [ ]:
import community
partition = community.best_partition(G, resolution = 1)

community_data = []

for community_id in set(partition.values()):
    nodes_in_community = [node for node, comm_id in partition.items() if comm_id == community_id]
    
    subgraph = G.subgraph(nodes_in_community)
    
    num_nodes = subgraph.number_of_nodes()
    num_edges = subgraph.number_of_edges()
    density = nx.density(subgraph)
    avg_degree = sum(dict(subgraph.degree()).values()) / num_nodes
    clustering_coefficient = nx.average_clustering(subgraph)
    partition2 = community.best_partition(subgraph)
    modularity = community.modularity(partition2, subgraph)
    edge_weights = [data['weight'] for _, _, data in subgraph.edges(data=True)]
    average_weight = sum(edge_weights) / len(edge_weights) if edge_weights else 0
    
    community_data.append((community_id, num_nodes, num_edges, density, avg_degree, clustering_coefficient, modularity, average_weight))

df_community = pd.DataFrame(community_data, columns=['Community_ID', 'Num_Nodes', 'Num_Edges', 'Density', 'Avg_Degree', 'Clustering_Coefficient', 'Modularity', 'Avg_Weight'])


In [ ]:
df_community.sort_values('Num_Nodes', ascending = False).sum()

In [ ]:
df_community_short = df_community.sort_values('Num_Nodes', ascending = False)[['Community_ID', 'Num_Nodes','Num_Edges', 'Density','Avg_Degree', 'Avg_Weight']]

In [ ]:
df_community_short['Community_ID'] = 'Group_' +df_community_short['Community_ID'].astype('str')

In [ ]:
df_community.sort_values('Num_Nodes', ascending = False).iloc[0:10]

In [ ]:
communities = {}
for node, community_id in partition.items():
    if community_id not in communities:
        communities[community_id] = []
    communities[community_id].append(node)

for community_id, nodes in communities.items():
    print(f"Community {community_id}: {len(nodes)}")

In [ ]:
pd.DataFrame(partition, columns=['Community_ID'])

In [ ]:
df_partition = pd.read_csv('Account_Community_Mapping_Refined.csv')

In [ ]:
df_partition 

In [ ]:
aque = pd.merge(df_partition, df_account[['User_ID', 'party']]).groupby(['Community_ID', 'party']).count().reset_index()

In [ ]:
df_community_short=pd.read_csv('Community_Stats.csv')

In [ ]:
df_community_short

In [ ]:
aque.loc[aque.Community_ID == 17].sum()

In [ ]:
df_partition2 = pd.merge(df_partition, df_account[['User_ID', 'party']])

In [ ]:
df_partition2.groupby(['Community_ID','party']).count().reset_index().sort_values('User_ID', ascending = False).iloc[0:20]

In [ ]:
df_partition2.groupby('Community_ID').count()

In [ ]:
df_partition2.loc[df_partition2.Community_ID ==17].sort_values('party').groupby('party').count()

In [ ]:
df_qqqq = df_partition2.loc[df_partition2.party != 'NonPolitical'].groupby(['Community_ID', 'party']).count().reset_index()

In [ ]:
df_partition2.loc[(df_partition2.Community_ID == 1) & 
                  (df_partition2.party == 'INC')]

In [ ]:
len(df_qqqq.loc[df_qqqq.party == 'INC'])

In [ ]:
for u, v, data in G.edges(data=True):
    if data['weight'] >= 1000:
        print(f"Edge ({u}, {v}): Weight = {data['weight']}")

In [ ]:
check_lable3 = df_partition.groupby('Community_ID').count().reset_index().sort_values('User_ID', ascending = False).iloc[0:9].Community_ID.tolist()

In [ ]:
df_check3 = df_partition[df_partition.Community_ID.isin(check_lable3)]

In [ ]:
df_check3 = pd.merge(df_check3, df_account[['User_ID', 'party']])

In [ ]:
df_check3_non_pol = df_check3.loc[df_check3.party == 'NonPolitical']

In [ ]:
df_check3_non_pol.groupby('Community_ID').count()

In [ ]:
def sample_accounts(group):
    return group.sample(min(len(group), 10))

sampled_df = df_check3_non_pol.groupby('Community_ID', group_keys=False).apply(sample_accounts)

sampled_df.reset_index(drop=True, inplace=True)



In [ ]:
sampled_df.sort_values('User_ID')

In [ ]:
df_pol_man1 = pd.read_csv('Political_Affiliation_Manual_Check1.csv')
df_pol_man2 = pd.read_csv('Political_Affiliation_Manual_Check2.csv')

In [ ]:
len(df_pol_man1.User_ID.astype('int').tolist() + df_pol_man2.User_ID.astype('int').tolist())

In [ ]:
sampled_df[sampled_df.User_ID.isin(df_pol_man1.User_ID.astype('int').tolist() + df_pol_man2.User_ID.astype('int').tolist())]

In [ ]:
sampled_df = pd.merge(sampled_df, df_account[['User_ID','UserDescription','ScreenName','UserFollowers','UserFriends','TotalStatus']])

In [ ]:
sampled_df = sampled_df.sample(frac = 1)

In [ ]:
sampled_df = sampled_df.reset_index(drop = True) 

In [ ]:
sampled_df 

In [ ]:
sampled_df3 = sampled_df[['User_ID', 'ScreenName', 'UserDescription', 'UserFollowers','UserFriends','TotalStatus']]

In [ ]:
sampled_df3['User_Link'] = 'https://twitter.com/i/user/' + sampled_df3['User_ID'].astype('str')